<a href="https://colab.research.google.com/github/ramesh-dev-code/openvino-projects/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello Image Classification

This basic introduction to OpenVINO™ shows how to do inference with an image classification models such as renset-50 (Tensorflow) and resnet-18 (PyTorch).

Refer to [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/) to download a wide range of pre-trained models and ready-to-run demos. For more information about how OpenVINO IR models are created, refer to the [TensorFlow to OpenVINO](../tensorflow-classification-to-openvino/tensorflow-classification-to-openvino.ipynb) tutorial.


#### Table of contents:

- [Imports](#Imports)
- [Download the Model and data samples](#Download-the-Model-and-data-samples)
- [Select inference device](#Select-inference-device)
- [Load the Model](#Load-the-Model)
- [Load an Image](#Load-an-Image)
- [Do Inference](#Do-Inference)



In [1]:
import platform

# Install openvino package
%pip install -q "openvino>=2023.1.0" opencv-python tqdm

if platform.system() != "Windows":
    %pip install -q "matplotlib>=3.4"
else:
    %pip install -q "matplotlib>=3.4,<3.7"



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 MB 6.6 MB/s eta 0:00:00


## Imports
[back to top ⬆️](#Table-of-contents:)


In [2]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import openvino as ov

# Fetch `notebook_utils` module
import requests

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)

open("notebook_utils.py", "w").write(r.text)

from notebook_utils import download_file

## Download the Model and data samples
[back to top ⬆️](#Table-of-contents:)


In [3]:
!omz_downloader --name resnet-50-tf -o ./artifacts

/bin/bash: line 1: omz_downloader: command not found


In [4]:
! omz_converter --name resnet-50-tf -o ./artifacts -d ./artifacts --precision FP16

/bin/bash: line 1: omz_converter: command not found


## Select inference device
[back to top ⬆️](#Table-of-contents:)

select device from dropdown list for running inference using OpenVINO

In [5]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

## Load the Model
[back to top ⬆️](#Table-of-contents:)


In [6]:
base_artifacts_dir = Path("./artifacts/public/resnet-50-tf/FP16/").expanduser()
file_name = "resnet-50-tf.xml"
model_xml_path = base_artifacts_dir / file_name
core = ov.Core()
model = core.read_model(model=model_xml_path)
compiled_model = core.compile_model(model=model, device_name=device.value)

output_layer = compiled_model.output(0)

RuntimeError: Exception from src/inference/src/cpp/core.cpp:90:
Check 'util::directory_exists(path) || util::file_exists(path)' failed at src/frontends/common/src/frontend.cpp:113:
FrontEnd API failed with GeneralFailure:
ir: Could not open the file: "artifacts/public/resnet-50-tf/FP16/resnet-50-tf.xml"



## Load an Image
[back to top ⬆️](#Table-of-contents:)


In [ ]:
# Download the image from the openvino_notebooks storage
data_dir = Path("./data").expanduser()
image_filename = data_dir / "t3.jpg"
# The MobileNet model expects images in RGB format.
image = cv2.cvtColor(cv2.imread(filename=str(image_filename)), code=cv2.COLOR_BGR2RGB)

# Resize to MobileNet image shape
input_image = cv2.resize(src=image, dsize=(224, 224))
#input_image = input_image.transpose(2, 0, 1)

# Reshape to model input shape.
input_image = np.expand_dims(input_image, 0)
plt.imshow(image);

## Do Inference
[back to top ⬆️](#Table-of-contents:)


In [ ]:
result_infer = compiled_model([input_image])[output_layer]
result_index = np.argmax(result_infer)

In [ ]:
imagenet_filename = download_file(
    "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/datasets/imagenet/imagenet_2012.txt",
    directory="data",
)

imagenet_classes = imagenet_filename.read_text().splitlines()
# The model description states that for this model, class 0 is a background.
# Therefore, a background must be added at the beginning of imagenet_classes.
imagenet_classes = ["background"] + imagenet_classes

In [ ]:
imagenet_classes[result_index]

## Conversion of PyTorch Model using OpenVINO model conversion API  

In [ ]:
import openvino as ov
import torch
from pathlib import Path
import torchvision
from torchvision.models import resnet18

# get default weights using available weights Enum for model
weights = torchvision.models.ResNet18_Weights.DEFAULT

# create model topology and load weights
model = resnet18(weights=weights)

# switch model to inference mode
model.eval()

# Convert model to openvino.runtime.Model object
ov_model = ov.convert_model(model, input=[[1, 3, 224, 224]])
prep = ov.preprocess.PrePostProcessor(ov_model)
prep.input("x").tensor().set_layout(ov.Layout("nchw"))
prep.input("x").preprocess().mean([123.675,116.28,103.53])
prep.input("x").preprocess().scale([58.395,57.12,57.375])
ov_model = prep.build()
base_artifacts_dir = Path("./artifacts/public/resnet-18-pytorch/ov_format/").expanduser()
ov_model_path = base_artifacts_dir / "resnet-18-pytorch.xml"
ov.save_model(ov_model,ov_model_path)

In [ ]:
ov_model

### Execute inference with PyTorch model

In [ ]:
from PIL import Image
import numpy as np
import torch
from pathlib import Path
import torchvision
from torchvision.models import resnet18

# get default weights using available weights Enum for model
weights = torchvision.models.ResNet18_Weights.DEFAULT

# create model topology and load weights
model = resnet18(weights=weights)

# switch model to inference mode
model.eval()

# Prepare Input Data
data_dir = Path("./data").expanduser()
image_filename = data_dir / "goldfish.jpg"
# The model expects images in RGB format.
image = Image.open(image_filename)

# Initialize the Weight Transforms
preprocess = weights.transforms()

# Apply it to the input image
img_transformed = preprocess(image)

# Add batch dimension to image tensor
input_tensor = img_transformed.unsqueeze(0)

imagenet_filename = data_dir / "imagenet_2012.txt"

# Perform model inference on input tensor
pt_infer_result = model(input_tensor)
result_index = np.argmax(pt_infer_result.detach().numpy())
imagenet_classes = imagenet_filename.read_text().splitlines()
imagenet_classes[result_index]

### Execute inference with the optimized model

In [ ]:
import openvino as ov
import cv2
import numpy as np
core = ov.Core()
base_artifacts_dir = Path("./artifacts/public/resnet-18-pytorch/ov_format/").expanduser()
ov_model_path = base_artifacts_dir / "resnet-18-pytorch.xml"
model_ov = core.read_model(model=ov_model_path)
compiled_model = core.compile_model(model=model_ov, device_name="CPU")
output_layer = compiled_model.output(0)

# Image Preprocessing
image = cv2.cvtColor(cv2.imread(str(image_filename)), cv2.COLOR_BGR2RGB)
input_image = cv2.resize(src=image, dsize=(224, 224))
input_image = input_image.transpose(2, 0, 1)
input_tensor_ov = np.expand_dims(input_image, 0)

# Display the preprocessed input image
#cv2.imshow("Input",image)
#cv2.waitKey(0)

# Execute inference on the compiled model
result_infer = compiled_model([input_tensor_ov])[output_layer]
result_index = np.argmax(result_infer)

# Get the predicted class from the ImageNet 2012 label file
imagenet_filename = data_dir / "imagenet_2012.txt"
imagenet_classes = imagenet_filename.read_text().splitlines()
print('Inference Result: ', imagenet_classes[result_index])

### Benchmark PyTorch Model Inference

In [ ]:
%%timeit

# Run model inference
model(input_tensor)

### Benchmark OpenVINO Model Inference

In [ ]:
%%timeit

# Run model inference
compiled_model([input_tensor_ov])[output_layer]